In [2]:
import spotipy
import math
import pandas as pd

In [3]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id = 'a3d051c55e7b473bb5663eadcbd6584a',
                                                      client_secret='3892e85b42554aaf963054e2c942daab')

In [4]:
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# Jai Wolf's ID
albums_foo = spotify.artist_albums(artist_id='24V5UY0nChKpnb1TBPJhCw', limit=50)

In [6]:
albums = filter(lambda a: a['album_type'] =='single', albums_foo['items'])

In [7]:
album_ids = []
album_names = []
album_date = []
for album in albums:
    album_ids.append(album['id'])
    album_names.append(album['name'])
    album_date.append(album['release_date'])

# Indexes to not consider
mask = [1, 2, 3, 5, 6, 7, 9, 10, 11]

album_n = []
album_id = []
album_d = []

for i in range(0, len(album_names)):
    if(i not in mask):
        album_n.append(album_names[i])
        album_id.append(album_ids[i])
        album_d.append(album_date[i])

In [8]:
# Create album names as IDs

In [9]:
album_replace = {}
album_date_ref = {}
for id, name in zip(album_id, album_n):
    album_replace[id] = name
for id, date in zip(album_id, album_d):
    album_date_ref[id] = date

In [10]:
features_required = ['energy', 'duration_ms', 'loudness', 'valence', 'acousticness', 'danceability', 'speechiness','key']

In [11]:
def get_track_features(songs, service):
    track_ids = [foo['id'] for foo in songs]
    chunk_size = 50
    num_chunks = int(math.ceil(len(track_ids) / float(chunk_size)))
    features_add = []
    for i in range(num_chunks):
        chunk_track_ids = track_ids[i*chunk_size:(i+1)*chunk_size]
        chunk_features = service.audio_features(tracks=chunk_track_ids)
        features_add.extend(chunk_features)
    
    features_df = pd.DataFrame(features_add)
    features_df = features_df[features_required]
    return features_df

In [12]:
songs_df_data = []
for album_i in album_id:
    #get songs in the album
    songs = spotify.album_tracks(album_id=album_i)['items']
    features = get_track_features(songs, spotify)
    features = features.values.tolist()
    #moods = [get_mood(foo['artists'][0]['name'], foo['name']) for foo in songs]
    for (track, feature) in zip(songs, features):
        t = track
        this_row = [t['id'], t['name'], album_i]       
        this_row.extend(feature)
        this_row.append(album_date_ref.get(album_i))
        songs_df_data.append(this_row)
    print(album_i)

50w4LlDe8QaW6aKdFopGLh
4hZjc57YQKXWLabnOZ5zku
4KVkQWwKyVGuJctOGczuO6
6dJAqemnQqc33N9uEzV9vX
4yrDQAqHKyqRRYOIlc13bo
47ufsRZskThlJZYLZiHno4
2zsjz9c784heHqP83ZFW0m
41JcySyB2ps5j4bt5LPReq
6SSxrc5bKgQKRAUBnfaO6j
0gFdd7aZWyDzhZZDNt2IFw
3XQjbVeRmkTjBbaWqtGFZe
2sIHxYSxsqg8q3rdN9647N
2mQD3ASLjLeFgxa9blLNJP
1D1MOzoKlsTbx0LuwYPJFW
16CCJo59jCko0476mRMmQu
0vVc7vyB035YeXeMWb9wUB
6y5UEDwbSAe0IylQpexNGF
4XfWzKytfZUpI2pQJXwL9c
0yOzkv7hjMuDI7DemPtdmm
4jPDQqOb389zAPYdUYpmVh
3WJeQAgxZ3D5y1O2DCbmik
6BjMA29wuTNuG8pz1WARo1
7CvJwnoSi3y4YKae5mDpbZ
5Ojm8NrCwdEI2cWVMArA60
3qoMadqfValFjOdwKh9TsL
1sTZdpb3v2PkYIP2yeFZPM
2afNG92zO2m0WT1FvU8PvA
627pIC5XCb0OBW95e6PJb1
4uRfYhBBaSX8N3amojmMBE
0AZUl7OzF7pt4EKRxHwHRm


In [13]:
import pandas as pd
songs_df = pd.DataFrame(songs_df_data, columns=['id', 'name', 'album', 'energy', 'duration_ms', 'loudness', 'valence', 'acousticness', 'danceability', 'speechiness', 'key', 'Date']).drop_duplicates(subset='id')

In [14]:
songs_df.replace(to_replace=album_replace, inplace=True)

In [15]:
songs_df.head(50)

,id,name,album,energy,duration_ms,loudness,valence,acousticness,danceability,speechiness,key,Date
0,0UVpLBGoS2lLM8F75yh1go,Better Apart (feat. Dresage),Better Apart (feat. Dresage),0.703,234987.0,-4.564,0.2560,0.013000,0.489,0.0260,4.0,2019-03-29
1,6fyfUncrRzeOOEcVYoDWT7,Your Way (feat. Day Wave),Your Way (feat. Day Wave),0.698,266160.0,-6.109,0.1230,0.003650,0.574,0.0277,7.0,2019-02-14
2,7ohJDePSTCjhYRLOd8Pwz4,Lost - Midnight Kids Remix,Lost Remixes,0.735,209714.0,-7.236,0.1890,0.138000,0.486,0.0308,6.0,2018-10-19
3,6yEm4os5PAllK19N6Q3nQ8,Lost - CRAY Remix,Lost Remixes,0.499,265200.0,-8.500,0.3780,0.175000,0.745,0.0442,6.0,2018-10-19
4,5SreLsy2YQW6C74D1onpth,Lost - Enschway Remix,Lost Remixes,0.504,201714.0,-8.571,0.2560,0.041100,0.528,0.0404,6.0,2018-10-19
5,2oj3zzAEbqlQozHL0OokJ3,Lost - Foxen Remix,Lost Remixes,0.831,248000.0,-3.184,0.6030,0.038600,0.540,0.0540,6.0,2018-10-19
6,6la2oViZ1nXIZ16AxMugk8,Lost - Wheezly Remix,Lost Remixes,0.714,204024.0,-8.403,0.4670,0.038600,0.334,0.1990,8.0,2018-10-19
7,11G8PZVKh233J7QUpEgHnf,Lost - Escape Remix,Lost Remixes,0.817,186677.0,-8.347,0.5530,0.040200,0.652,0.0365,1.0,2018-10-19
8,39JRtAMrNpcMAZ2LxC69df,Lost - Dillon Nathaniel Remix,Lost Remixes,0.865,267097.0,-7.939,0.6380,0.007890,0.785,0.0872,6.0,2018-10-19
9,6uDQIjSAnbEr0tNAH6J9cr,Lost - ATTLAS Remix,Lost Remixes,0.917,229111.0,-3.852,0.5190,0.015800,0.585,0.0451,3.0,2018-10-19


In [16]:
songs_df['remix_flag'] = songs_df.apply(lambda row: 1 if row['name'].find('Remix') != -1 or row['album'].find('Remix') != -1 else 0, axis = 1)
songs_df.head(50)

,id,name,album,energy,duration_ms,loudness,valence,acousticness,danceability,speechiness,key,Date,remix_flag
0,0UVpLBGoS2lLM8F75yh1go,Better Apart (feat. Dresage),Better Apart (feat. Dresage),0.703,234987.0,-4.564,0.2560,0.013000,0.489,0.0260,4.0,2019-03-29,0
1,6fyfUncrRzeOOEcVYoDWT7,Your Way (feat. Day Wave),Your Way (feat. Day Wave),0.698,266160.0,-6.109,0.1230,0.003650,0.574,0.0277,7.0,2019-02-14,0
2,7ohJDePSTCjhYRLOd8Pwz4,Lost - Midnight Kids Remix,Lost Remixes,0.735,209714.0,-7.236,0.1890,0.138000,0.486,0.0308,6.0,2018-10-19,1
3,6yEm4os5PAllK19N6Q3nQ8,Lost - CRAY Remix,Lost Remixes,0.499,265200.0,-8.500,0.3780,0.175000,0.745,0.0442,6.0,2018-10-19,1
4,5SreLsy2YQW6C74D1onpth,Lost - Enschway Remix,Lost Remixes,0.504,201714.0,-8.571,0.2560,0.041100,0.528,0.0404,6.0,2018-10-19,1
5,2oj3zzAEbqlQozHL0OokJ3,Lost - Foxen Remix,Lost Remixes,0.831,248000.0,-3.184,0.6030,0.038600,0.540,0.0540,6.0,2018-10-19,1
6,6la2oViZ1nXIZ16AxMugk8,Lost - Wheezly Remix,Lost Remixes,0.714,204024.0,-8.403,0.4670,0.038600,0.334,0.1990,8.0,2018-10-19,1
7,11G8PZVKh233J7QUpEgHnf,Lost - Escape Remix,Lost Remixes,0.817,186677.0,-8.347,0.5530,0.040200,0.652,0.0365,1.0,2018-10-19,1
8,39JRtAMrNpcMAZ2LxC69df,Lost - Dillon Nathaniel Remix,Lost Remixes,0.865,267097.0,-7.939,0.6380,0.007890,0.785,0.0872,6.0,2018-10-19,1
9,6uDQIjSAnbEr0tNAH6J9cr,Lost - ATTLAS Remix,Lost Remixes,0.917,229111.0,-3.852,0.5190,0.015800,0.585,0.0451,3.0,2018-10-19,1


In [17]:
songs_original_df = songs_df[songs_df['remix_flag']==0]

In [18]:
songs_remixes_df = songs_df[songs_df['remix_flag']==1]

In [19]:
# Let's deal with remixes later

In [20]:
# Originals only
# libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
songs_original_df.describe()

,energy,duration_ms,loudness,valence,acousticness,danceability,speechiness,key,remix_flag
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.0
mean,0.758962,250921.923077,-3.552808,0.160419,0.124874,0.425423,0.042981,4.192308,0.0
std,0.148598,57603.002599,2.034615,0.097401,0.192199,0.075568,0.019908,3.666270,0.0
min,0.325000,95500.000000,-7.031000,0.035200,0.003650,0.265000,0.026000,0.000000,0.0
25%,0.707250,227951.250000,-5.512500,0.042450,0.024450,0.410000,0.032000,0.000000,0.0
50%,0.814000,248471.000000,-2.507000,0.182000,0.053150,0.432000,0.033400,4.000000,0.0
75%,0.842750,298667.000000,-1.821250,0.210000,0.129525,0.477250,0.049675,6.000000,0.0
max,0.938000,318000.000000,-1.701000,0.334000,0.795000,0.574000,0.114000,10.000000,0.0


In [22]:
# Sorting the data by date
songs_original_df = songs_original_df.sort_values(by='Date', ascending=True)

In [23]:
songs_original_df = songs_original_df.drop_duplicates(subset='name', keep="last")

In [24]:
kindered_spirits_df = songs_original_df[songs_original_df['album']=='Kindred Spirits']

In [25]:
with open('x.txt', 'r') as file:
    kslist = file.read().split('\n')

In [26]:
ks=[]
for item in kslist:
    x = item.split(',')
    ks.append(x)
    print(x)

['This Space in My Heart Is for You', '1']
['Indian Summer', '2']
['Gravity', '3']
["Like It's Over", '4']
['The World Is Ours', '5']
['Drive', '6']


In [27]:
def find_order(row):
    for item in ks:
        if row['name'].upper().find(item[0].upper()) != -1:
            return item[1]
    return 99
kindered_spirits_df['order'] = kindered_spirits_df.apply(lambda row: find_order(row), axis = 1)


C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
kindered_spirits_df.head(5)

,id,name,album,energy,duration_ms,loudness,valence,acousticness,danceability,speechiness,key,Date,remix_flag,order
46,6DlEaldMTn1lqL9S0wFtAv,Drive,Kindred Spirits,0.853,298667.0,-1.822,0.211,0.0976,0.433,0.0326,0.0,2016-11-18,0,6
44,4jrfV0jdUwRKVWvc77PG4F,Like It's Over,Kindred Spirits,0.802,318000.0,-1.708,0.182,0.1410,0.408,0.0330,0.0,2016-11-18,0,4
43,0hhQi5rzChpYnho9GsxYTJ,Gravity,Kindred Spirits,0.652,228480.0,-7.024,0.333,0.0307,0.456,0.0296,10.0,2016-11-18,0,3
40,0TMd00Vk2lYaEorXuta9tk,Drive (feat. Chain Gang of 1974),Kindred Spirits,0.849,298667.0,-1.821,0.203,0.0976,0.432,0.0318,0.0,2016-11-18,0,6
39,0YDSEyBzPsuQzkuc9Bu8iR,The World Is Ours,Kindred Spirits,0.938,222400.0,-2.926,0.259,0.0534,0.484,0.0408,4.0,2016-11-18,0,5


In [29]:
# Understanding the progression of the album
kindered_spirits_df = kindered_spirits_df.drop_duplicates(subset='order', keep="last")

In [30]:
kindered_spirits_df.to_csv('ks.csv')

In [32]:
songs_original_df.to_csv('songs_original.csv')

In [38]:
songs_original_df['Date_int'] = songs_original_df.apply(lambda row: row['Date'].replace('-',''), axis = 1)

In [42]:
songs_original_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 46 to 0
Data columns (total 15 columns):
id              14 non-null object
name            14 non-null object
album           14 non-null object
energy          14 non-null float64
duration_ms     14 non-null float64
loudness        14 non-null float64
valence         14 non-null float64
acousticness    14 non-null float64
danceability    14 non-null float64
speechiness     14 non-null float64
key             14 non-null float64
Date            14 non-null object
remix_flag      14 non-null int64
Date_int        14 non-null object
new_date        14 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(8), int64(1), object(5)
memory usage: 1.8+ KB


In [40]:
songs_original_df['new_date'] = pd.to_datetime(songs_original_df['Date_int'].astype(str), format='%Y%m%d')

In [ ]:
songs_original_df.to_csv('songs_df_time_fixed.csv')